In [3]:
import requests
import json
import time
import datetime
import pickle
from collections import Counter

In [19]:
#Incluir la key personal
#key = "xxxx"

In [50]:
#Función para obtener dos listas de fechas
#La primera lista contiene el primer día de cada mes
#Y la segunda el último día de cada mes
#Los argumentos de la función son el mes y año (incluido) donde deben comenzar las listas y el mes y año (no incluido) donde deben terminar

def inicio_final(año_inicial, mes_inicial, año_final, mes_final):
    
    fechas_iniciales = []
    fechas_finales = []
    
    año = año_inicial
    mes = mes_inicial
    
    while (año != año_final) or (mes != mes_final):
        inicio = datetime.datetime(año, mes, 1)
        fecha_inicial = inicio.strftime("%Y-%m-%d")
        fechas_iniciales.append(fecha_inicial)
        mes  += 1
        if mes == 13:
            mes = 1
            año += 1
        final = datetime.datetime(año, mes, 1) - datetime.timedelta(days = 1)
        fecha_final = final.strftime("%Y-%m-%d")
        fechas_finales.append(fecha_final)
        
    return fechas_iniciales, fechas_finales

In [58]:
#Para este proyecto se va a utilizar la serie temporal que va de enero 2015 hasta febrero 2021
#En el argumento de la función se pone como fecha final marzo 2021, ya que este mes no está incluido

fechas = inicio_final(2015, 1, 2021, 3)

In [62]:
#Con esta función se obtienen los datos meteorológicos de la AEMET
#El argumento de la función son las listas de fechas creadas anteriormente

def datos_aemet(fechas):
    url1 = "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/"
    url2 = "T00%3A00%3A00UTC/fechafin/"
    url3 = "T23%3A59%3A59UTC/todasestaciones"
    
    lista_datos = []
    
    for i in range(len(fechas[0])):
        url = url1 + fechas[0][i] + url2 + fechas[1][i] + url3
        
        #Se hace la consulta a la AEMET
        querystring = {"api_key" : key}
        headers = {
            'cache-control': "no-cache"
            }
        response = requests.request("GET", url, headers=headers, params=querystring)

        #Se transforma la respuesta a .json y se obtiene la dirección web que contiene los datos que buscamos
        response = response.json()
        web = response["datos"]
        
        #Se obtienen los datos buscados desde la dirección web 
        datos = requests.get(web)
        datos = datos.json()
        lista_datos.append(datos)
        time.sleep(10)
    
    return lista_datos

In [1]:
#lista_datos es una lista de 74 elementos o sublistas
#Cada una de estas sublistas posee los datos de todas las estaciones meteorológicas de un mes (de enero 2015 a febrero 2021, ambos inclusive)
#La información de cada estación meteorológica se encuentran en diccionarios
#Se trata de una variable muy pesada, con gran cantidad de datos

lista_datos = datos_aemet(fechas)

In [5]:
#Un ejemplo de cómo son los diccionarios que contienen la información de cada estación meteorológica

lista_datos[0][0]

{'fecha': '2015-01-01',
 'indicativo': '4358X',
 'nombre': 'DON BENITO',
 'provincia': 'BADAJOZ',
 'altitud': '273',
 'tmed': '5,6',
 'prec': '0,0',
 'tmin': '-2,6',
 'horatmin': '07:40',
 'tmax': '13,7',
 'horatmax': '15:30',
 'dir': '36',
 'velmedia': '0,8',
 'racha': '3,6',
 'horaracha': '13:40',
 'sol': '8,4',
 'presMax': '1004,2',
 'horaPresMax': 'Varias',
 'presMin': '999,9',
 'horaPresMin': '02'}

In [66]:
#Se guarda en local la información de las estaciones meteorológicas

fh = open("aemet_2015_2021.pkl", "wb")
pickle.dump(lista_datos, fh)
fh.close()

In [4]:
#Se carga la información de las estaciones meteorológicas

fh = open("aemet_2015_2021.pkl", "rb")
lista_datos = pickle.load(fh)
fh.close()

<function BufferedReader.close>

In [7]:
#Se crea un diccionario que contiene como llaves el indicativo (como un id) de cada estación meteorológica
#Y como valores la provincia en la que se encuentra cada una de ellas

estaciones_provincia = {}

for i in lista_datos:
    for j in i:
        if j["provincia"] not in estaciones_provincia:
            estaciones_provincia[j["indicativo"]] = j["provincia"]
        else:
            pass
    
estaciones_provincia

{'4358X': 'BADAJOZ',
 '4220X': 'CIUDAD REAL',
 'C447A': 'STA. CRUZ DE TENERIFE',
 '6106X': 'MALAGA',
 '9698U': 'LLEIDA',
 '4410X': 'BADAJOZ',
 '1331A': 'ASTURIAS',
 '1690A': 'OURENSE',
 '0002I': 'TARRAGONA',
 '8489X': 'CASTELLON',
 '8025': 'ALICANTE',
 'C449C': 'STA. CRUZ DE TENERIFE',
 '9784P': 'HUESCA',
 '2331': 'BURGOS',
 '9434': 'ZARAGOZA',
 '1109': 'CANTABRIA',
 '6293X': 'ALMERIA',
 '1212E': 'ASTURIAS',
 '3094B': 'CUENCA',
 '1059X': 'BIZKAIA',
 '1207U': 'ASTURIAS',
 '6367B': 'ALMERIA',
 '7228': 'MURCIA',
 '3168C': 'GUADALAJARA',
 '7002Y': 'MURCIA',
 'C649I': 'LAS PALMAS',
 '3191E': 'MADRID',
 '0016A': 'TARRAGONA',
 '8058X': 'VALENCIA',
 '0367': 'GIRONA',
 '7247X': 'ALICANTE',
 '3200': 'MADRID',
 '4103X': 'CIUDAD REAL',
 '3175': 'MADRID',
 '3129': 'MADRID',
 '4067': 'TOLEDO',
 'C689E': 'LAS PALMAS',
 '1014A': 'GIPUZKOA',
 '7031X': 'MURCIA',
 '7209': 'MURCIA',
 '3519X': 'CACERES',
 '2755X': 'ZAMORA',
 '8175': 'ALBACETE',
 '9244X': 'ZARAGOZA',
 '6277B': 'ALMERIA',
 '9170': 'LA RIOJA'

In [8]:
#Se guarda el diccionario que se acaba de crear, ya que se va a usar en el notebook de electricidad

fh = open("estaciones_provincia.pkl", "wb")
pickle.dump(estaciones_provincia, fh)
fh.close()